In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
df=pd.read_csv("/content/Sentiment.csv")

In [3]:
df.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [4]:
df=df[["text","sentiment"]]

In [5]:
df.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [6]:
df.isna().sum()

text         0
sentiment    0
dtype: int64

In [7]:
df.shape

(13871, 2)

In [8]:
df.duplicated().sum()

3304

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
df.shape

(10567, 2)

In [13]:
df.drop(df[df['sentiment'] == 'Neutral'].index,inplace=True)

In [14]:
df.shape

(7838, 2)

In [ ]:
#Print the total number of positive and negative sentiments. 

In [19]:
print("Number of positive sentiments",len(df[df.sentiment=="Positive"]))
print("Number of negative sentiments",len(df[df.sentiment=="Negative"]))

Number of positive sentiments 1709
Number of negative sentiments 6129


In [20]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
corpus=[]
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()

In [22]:
for i in range(len(df['text'])):
  review=re.sub('[^A-Za-z]',' ',df['text'].iloc[i])
  review=review.lower()
  review=review.split()
  rev=[stemmer.stem(word) for word in review if word not in set(stopwords.words('english'))]
  rev=' '.join(rev)
  corpus.append(rev)

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [24]:
x=cv.fit_transform(corpus).toarray()
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [25]:
y=df['sentiment']
y

1        Positive
3        Positive
4        Positive
5        Positive
6        Negative
           ...   
13845    Negative
13847    Positive
13857    Negative
13859    Positive
13867    Positive
Name: sentiment, Length: 7838, dtype: object

In [18]:
#Build a sequential LSTM model to predict positive and negative sentiments.

In [37]:
tweet = df.text.values

In [38]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweet)

In [39]:
encoded_docs = tokenizer.texts_to_sequences(tweet)

In [40]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [41]:
reviews = df['text'].values
labels = df['sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [42]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels)


In [43]:
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [45]:
from tensorflow import keras
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          300000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [46]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
166/166 [==============================] - 58s 306ms/step - loss: 0.5081 - accuracy: 0.7803 - val_loss: 0.4118 - val_accuracy: 0.8163
Epoch 2/5
166/166 [==============================] - 45s 273ms/step - loss: 0.3373 - accuracy: 0.8609 - val_loss: 0.3969 - val_accuracy: 0.8248
Epoch 3/5
166/166 [==============================] - 42s 256ms/step - loss: 0.2591 - accuracy: 0.8962 - val_loss: 0.4343 - val_accuracy: 0.8350
Epoch 4/5
166/166 [==============================] - 43s 256ms/step - loss: 0.2043 - accuracy: 0.9181 - val_loss: 0.5174 - val_accuracy: 0.8265
Epoch 5/5
166/166 [==============================] - 43s 256ms/step - loss: 0.1583 - accuracy: 0.9422 - val_loss: 0.5804 - val_accuracy: 0.8044


In [47]:
um_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
166/166 [==============================] - 44s 265ms/step - loss: 0.1255 - accuracy: 0.9539 - val_loss: 0.7075 - val_accuracy: 0.8214
Epoch 2/5
166/166 [==============================] - 44s 264ms/step - loss: 0.0990 - accuracy: 0.9648 - val_loss: 0.7662 - val_accuracy: 0.8044
Epoch 3/5
166/166 [==============================] - 44s 265ms/step - loss: 0.0859 - accuracy: 0.9688 - val_loss: 0.9245 - val_accuracy: 0.8027
Epoch 4/5
166/166 [==============================] - 44s 266ms/step - loss: 0.0734 - accuracy: 0.9745 - val_loss: 0.9398 - val_accuracy: 0.8044
Epoch 5/5
166/166 [==============================] - 45s 270ms/step - loss: 0.0563 - accuracy: 0.9777 - val_loss: 1.1172 - val_accuracy: 0.8061


In [48]:
sentence = ['He is a great leader','He is a terrible leader.' ]
# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)
# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)
# Get labels based on probability 1 if p>= 0.5 else 0
prediction = model.predict(padded)
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
for i in range(len(sentence)):
    print(sentence[i])
    if pred_labels[i] == 1:
        s = 'Positive'
    else:
        s = 'Negative'
    print("Predicted sentiment : ",s)

He is a great leader
Predicted sentiment :  Positive
He is a terrible leader.
Predicted sentiment :  Positive
